# Allegro ClearML logging example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/allegroai/clearml/blob/master/examples/reporting/jupyter_logging_example.ipynb)

This example introduces ClearML [Logger](https://clear.ml/docs/latest/docs/fundamentals/logger) functionality. Logger is the ClearML console log and metric interface.

You can find more reporting examples [here](https://github.com/allegroai/clearml/tree/master/examples/reporting).

In [ ]:
# If you don't have ClearML installed then uncomment this line
# !pip install clearml

### Create a new Task
Create a new Task and get a Logger object for the Task.

To create a new Task object, call the `Task.init` method providing it with `project_name` (the project name for the experiment) and `task_name` (the name of the experiment). When `Task.init` executes, a link to the Web UI Results page for the newly generated Task will be printed, and the Task will be updated in real time in the ClearML demo server.

You can read about the `Task` class in the docs [here](https://clear.ml/docs/latest/docs/fundamentals/task).

After the Task is created, get a Logger for it.

In [ ]:
import numpy as np

from clearml import Task

# Start a new task
task = Task.init(project_name="Colab notebooks", task_name="Explicit Logging")

# Get the task logger,
# You can also call Task.current_task().get_logger() from anywhere in your code.
logger = task.get_logger()

### Explicit scalar logging
Use the [Logger.report_scalar](https://clear.ml/docs/latest/docs/references/sdk/logger#report_scalar) method to explicitly log scalars. Scalar plots appear in the Web UI, Results > Scalars tab.

In [ ]:
# report two scalar series on the same graph
for i in range(10):
    logger.report_scalar("unified graph", "series A", iteration=i, value=1./(i+1))
    logger.report_scalar("unified graph", "series B", iteration=i, value=10./(i+1))

# report two scalar series on two different graphs
for i in range(10):
    logger.report_scalar("graph A", "series A", iteration=i, value=1./(i+1))
    logger.report_scalar("graph B", "series B", iteration=i, value=10./(i+1))

### Explicit logging of other data

You can log other data and report the data in a variety of plot types, including histograms, confusion matrices, 2D and 3D scatter diagrams, and surface diagrams. They appear in the Results > Plots tab.

For information about the methods to report each type of plot, see the [Logger](https://clear.ml/docs/latest/docs/references/sdk/logger) module.


In [ ]:
iteration = 100

# report a single histogram
histogram = np.random.randint(10, size=10)
logger.report_histogram(
    "single_histogram",
    "random histogram",
    iteration=iteration,
    values=histogram,
    xaxis="title x",
    yaxis="title y",
)

# report a two histograms on the same plot
histogram1 = np.random.randint(13, size=10)
histogram2 = histogram * 0.75
logger.report_histogram(
    "two_histogram",
    "series 1",
    iteration=iteration,
    values=histogram1,
    xaxis="title x",
    yaxis="title y",
)
logger.report_histogram(
    "two_histogram",
    "series 2",
    iteration=iteration,
    values=histogram2,
    xaxis="title x",
    yaxis="title y",
)

# report confusion matrix
confusion = np.random.randint(10, size=(10, 10))
logger.report_matrix(
    "example_confusion",
    "ignored",
    iteration=iteration,
    matrix=confusion,
    xaxis="title X",
    yaxis="title Y",
)

scatter2d = np.hstack(
    (np.atleast_2d(np.arange(0, 10)).T, np.random.randint(10, size=(10, 1)))
)
# report 2d scatter plot with markers
logger.report_scatter2d(
    "example_scatter",
    "series_lines+markers",
    iteration=iteration,
    scatter=scatter2d,
    xaxis="title x",
    yaxis="title y",
    mode='lines+markers'
)

# report 3d surface
surface = np.random.randint(10, size=(10, 10))
logger.report_surface(
    "example_surface",
    "series1",
    iteration=iteration,
    matrix=surface,
    xaxis="title X",
    yaxis="title Y",
    zaxis="title Z",
)

# report 3d scatter plot
scatter3d = np.random.randint(10, size=(10, 3))
logger.report_scatter3d(
    "example_scatter_3d",
    "series_xyz",
    iteration=iteration,
    scatter=scatter3d,
    xaxis="title x",
    yaxis="title y",
    zaxis="title z",
)

### Explicit debug samples reporting

Explicitly report debug samples, including images, audio, and video.

#### Downloading the files

We use StorageManager to download a local copy of the files. You can use it immediately. Just provide the URL. Cache is enabled by default for all downloaded remote URLs/files.

For more information, you can read about the storage manager [here](https://clear.ml/docs/latest/docs/references/sdk/storage).

In [ ]:
from clearml.storage import StorageManager
image_local_copy = StorageManager.get_local_copy(
    remote_url="https://pytorch.org/tutorials/_static/img/neural-style/picasso.jpg",
    name="picasso.jpg"
)

print("Image location: {}".format(image_local_copy))

video_local_copy = StorageManager.get_local_copy(
    remote_url="https://test-videos.co.uk/vids/bigbuckbunny/mp4/h264/720/Big_Buck_Bunny_720_10s_1MB.mp4",
    name="Big_Buck_Bunny_720_10s_1MB.mp4"
)

print("Video location: {}".format(video_local_copy))

audio_local_copy = StorageManager.get_local_copy(
    remote_url="https://www2.cs.uic.edu/~i101/SoundFiles/PinkPanther30.wav",
    name="PinkPanther30.wav"
)

print("Audio location: {}".format(audio_local_copy))


#### Report images and media

Use [Logger.report_image](https://clear.ml/docs/latest/docs/references/sdk/logger#report_image) and [Logger.report_media](https://clear.ml/docs/latest/docs/references/sdk/logger#report_media) to report the downloaded samples. The debug samples appear in the Results > Debug Samples tab.

In [ ]:
logger.report_image("image", "image from url", iteration=100, local_path=image_local_copy)

# Image can be uploaded via 'report_media' too
# report video, an already uploaded video media (url)
logger.report_media(
    'video', 'big bunny', iteration=1, local_path=video_local_copy)


# This will actually use the cache and will not download the file again.
audio_local_copy_cache = StorageManager.get_local_copy(
    remote_url="https://www2.cs.uic.edu/~i101/SoundFiles/PinkPanther30.wav",
    name="PinkPanther30.wav"
)

# report audio, report an already uploaded audio media (url)
logger.report_media(
    'audio', 'pink panther', iteration=1, local_path=audio_local_copy)

# reporting html from url to debug samples section
logger.report_media("html", "url_html", iteration=1, url="https://clear.ml/docs/latest/docs/index.html")

### Explicit text logging
Use [Logger.report_text](https://clear.ml/docs/latest/docs/references/sdk/logger#report_text) to log text message. They appear in Results > Log.

In [ ]:
# report text
logger.report_text("hello, this is plain text")


### Flushing the reports

Reports are flushed in the background every couple of seconds, and at the end of the process execution.

Or, flush the Logger by calling [Logger.flush](https://clear.ml/docs/latest/docs/references/sdk/logger#flush).

In [ ]:
logger.flush()